In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
from armscan_env import config
from armscan_env.volumes.loading import load_sitk_volumes, resize_sitk_volume

config = config.get_config()

In [ ]:
volumes = load_sitk_volumes(normalize=False)

In [ ]:
normalized_volumes = resize_sitk_volume(volumes)

In [ ]:
print(volumes[5].GetSize())
print(normalized_volumes[1].GetSize())

In [ ]:
volume = normalized_volumes[6]
array = sitk.GetArrayFromImage(volume)
print(f"Slice value range: {np.min(array)} - {np.max(array)}")

In [ ]:
from armscan_env.clustering import TissueClusters
from armscan_env.envs.rewards import anatomy_based_rwd
from armscan_env.envs.state_action import ManipulatorAction
from armscan_env.util.visualizations import show_clusters

action = ManipulatorAction(rotation=(-3, 0), translation=(0, 178))
x_size_2, y_size_2, z_size_2 = (
    sz * sp for sz, sp in zip(volume.GetSize(), volume.GetSpacing(), strict=True)
)
extent_xy_2 = (0, x_size_2, y_size_2, 0)

spacing = volume.GetSpacing()
plt.imshow(array[33, :, :], extent=extent_xy_2)

o = volume.GetOrigin()
x_dash = np.arange(x_size_2)
b = action.translation[1]
y_dash = x_dash * np.tan(np.deg2rad(action.rotation[0])) + b
plt.plot(x_dash, y_dash, linestyle="--", color="red")

plt.show()

In [ ]:
slice_shape = (volume.GetSize()[0], volume.GetSize()[2])
sliced_volume = volume.get_volume_slice(
    slice_shape=slice_shape,
    action=action,
)
sliced_img = sitk.GetArrayFromImage(sliced_volume)
cluster = TissueClusters.from_labelmap_slice(sliced_img.T)
show_clusters(cluster, sliced_img.T, aspect=4)
plt.show()

In [ ]:
reward = anatomy_based_rwd(cluster)
print(reward)